In [7]:
import requests
from lxml import etree
import webbrowser
import time
import random as rn
import re
import urllib
import urllib.request
from bs4 import BeautifulSoup
import jieba
import jieba.analyse

In [17]:
歡迎 = ['哈囉！', '嗨！']
回歡迎 = ['怎麼了嗎？', '有什麼事嗎？', '要聊什麼呀？', '怎麼了嗎？']

吃東西 = ['吃', '肚子餓', '餐廳', '小吃', '餓']

喝東西 = ['喝', '口渴', '飲料']

時間 = ['時間', '幾點', '日期', '幾月', '幾日', '星期']

天氣 = ['天氣', '下雨', '太陽', '冷', '熱']

好心情 = ['開心', '快樂', '爽', '欣喜', '愉悅', '愉快']
回好心情 = ['還不錯呀！', '恭喜！', '聽起來不錯', '厲害唷！', '真為你高興！']

壞心情 = ['不開心', '不爽', '糟透了', '不好', '好累', '糟', '煩', '哭哭', '難過', '傷心', '心痛']
回壞心情 = ['拍拍', '一起加油', '沒事拉！', '讓它過去就好了', '別放在心上！']

繼續 = ['要再聊聊嗎？', '多講點', '還有嗎？', '然後呢？', '還有嗎？']

感覺 = ['如何', '你呢', '你覺得']
回感覺 = ['還好', '還可以', '尚可', '還可以接受']

感謝 = ['謝', '感謝']
回感謝 = ['不會拉！', '能幫到你很開心！']

離開 = ['掰', '下次見', '下線', '再見', '不聊']
回離開 = ['下次見', '下次聊', '掰掰', '很高興認識你！']

敷衍 = ['恩恩', 'XD', '好喔！', '喔！', 'www', 'ＱＱ', '哈哈！', '喔喔！', 'Aっ豆...']

In [18]:
def 問好():
    t = time.localtime(time.time())
    if(t[3]<11 and t[3]>=5):
        return '早安!'
    elif(t[3]>=11 and t[3]<14):
        return '午安!'
    elif(t[3]>=14 and t[3]<16):
        return '下午好!'
    elif(t[3]>=16 and t[3]<23):
        return '晚上好!'
    else:
        return '好晚了!怎麼還不快睡覺!'

In [22]:
# 政大吃什麼
餐點 = {'早午餐':['here cafe', '小公寓', '享窩'],
      '簡餐':['世界大同', '番茄紅', '蝴蝶漾', '葛莉絲', '口甲麵騎士', '金色漁家', '意鼎'],
      '義大利麵':['米塔', '多果', 'go cafe', '里克', 'here cafe', '發現', '享窩', '蝴蝶漾', 'upstair', '鄰居家', '提洛斯'],
      '燉飯':['米塔', '里克', 'here cafe', '發現', '蝴蝶漾', 'upstair', '鄰居家', '提洛斯'],
      '定食':['東京小城', '金鰭', '佐世保', '湯饌', '吉野家', '丼bar', '加賀屋', '樂山', '大正亭'],
      '生魚片':['加賀屋', '大正亭', '東京小城', '金鰭', '佐世保'],
      '越':['華越', '滿福越南料理', '政大越南小吃'],
      '泰':['小曼谷'],
      '炒飯':['敏忠', '廢墟', '波波恰恰', '美香味', '來來', '左撇子'],
      '燴飯':['敏忠', '四五大街', '左撇子', '來來'],
      '炒麵':['廢墟', '敏忠', '來來', '意鼎', '季旭小吃'],
      '乾麵':['京華小館', '甘泉魚麵', 'my麵屋', '口甲麵騎士', '悅來', '飽飽食府', '玉口香', '意鼎', '季旭小吃'],
      '便當':['蘭亭', '扒飯', '美香味', '楊記', '季旭小吃'],
      '麵包':['麵包大亨', '廣奇來', '米塔'],
      '海南雞飯':['私房麵', '波波恰恰'],
      '湯麵':['甘泉魚麵', '京華小館', 'my麵屋', '私房麵', '波波恰恰', '悅來', '飽飽食府', '玉口香', '八方雲集', '吃的小館', '麻咾王拉麵', '金鰭', '東京小城', '飯粥哥', '舅媽的店', '季旭小吃'],
      '東南亞':['華越', '波波恰恰', '私房麵', '滇味廚房', '滿福越南料理', '政大越南小吃'],
      '可麗餅':['屈臣氏旁邊的可麗餅'],
      '飯糰':['傳香', '古早味', '口福'],
      '蛋餅': ['小貓咪', '古早味', '口福', '德合香', '美味派', '戴記', '比司多'],
      '三明治':['享窩', '德合香', '摩斯', '米塔', '口福', '小貓咪', '星巴克', '路易莎', '谷庄茶奶', '美味派', '比司多'],
      '美式':['juicy bun', 'upstair'],
      '合菜':['四川', '巴東蜀味', '敘緣小館', '金色漁家'],
      '滷味':['丐幫', '吃的小館', '健康滷味', '愛心滷味', '京城滷味'],
      '韓':['東萊品', '韓大佬', '阿里郎', '高句麗'],
      '火鍋':['8鍋', '大呼過癮', 'shabu sen'],
      '鍋貼':['八方雲集'],
      '麵線':['展臭豆腐麵線', '舅媽的店', '119巷勝'],
      '臭豆腐':['展臭豆腐麵線'],
      '燒臘':['殺人魔王'],
      '速食':['麥當勞', '摩斯'],
      '素食':['素還真', '愛家蔬食便當'],
      '漢堡':['juicy bun', '麥當勞', '摩斯', '德合香', '小貓咪', 'upstair', '谷庄茶奶', '美味派'],
      '雞排':['政大烤場', '派克'],
      '薯條':['麥當勞', '摩斯', 'juicy bun', 'upstair'],
      '炸雞':['麥當勞', '摩斯'],
      '沙拉':['juicy bun', '摩斯', '麥當勞', '鄰居家', 'upstair', 'subway', 'here cafe'],
      '餅':['東北大餅'],
      '帕尼尼':['here cafe', '小公寓', '路易莎', '鄰居家', '享窩'],
      '蛋捲':['juicy bun', '鄰居家', 'upstair', '路易莎'],
      '潛艇堡':['subway'],
      '水餃':['八方雲集', '悅來', '飽飽食府'],
      '牛肉麵':['飽飽食府', '悅來'],
      '自助餐':['香香', '佳味', '大稻埕'],
      '石鍋拌飯':['高句麗', '韓大佬', '阿里郎', '多果'],
      '咖哩飯':['東京小城', '蝴蝶漾', '119巷勝'],
      '鹹酥雞':['政大烤場'],
      '炸魷魚':['政大烤場'],
      '烤':['政大烤場', '甄燒食堂'],
      '餛飩':['玉口香'],
      '羹':['羹大王', '舅媽的店', '麥側旁的羊肉羹魷魚羹'],
      '滷肉飯':['阿義師', '政大越南小吃', '羹大王', '京華小館', '廢墟', '私房麵', '舅媽的店'],
      '拉麵':['麻咾王拉麵'],
      'pizza':['鄰居家'],
      '粥':['飯粥哥'],
      '壽司':['東京小城', '金鰭', '加賀屋'],
      '貝果':['cow banana'],
      '蛋糕':['米塔', '廣奇來', '麵包大亨', '路易莎', '星巴克', '5cafe', 'joline'],
      '豆花':['明池'],
      '西米露':['明池'],
      '仙草':['明池', '廢墟前的台灣傳統客家'],
      '咖啡店':['星巴克', '路易莎', '道南館', 'prowin paris coffee', 'lee brothers 44 bisto', '5cafe'],
      '水煎包':['老麵水煎包']}
飲料 = {'奶茶':['龍角', '五十嵐', 'Comebuy', '政大茶亭', '星巴克', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶'],
      '咖啡':['龍角', '五十嵐', 'Comebuy', '星巴克', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶', '5 Cafe'],
      '鮮奶茶':['龍角', 'Cow banana', '找茶道', '谷庄茶奶'],
      '果茶':['龍角', 'Comebuy', '政大茶亭', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶'],
      '果汁':['Comebuy', '政大茶亭', '星巴克', '咖啡大亨'],
      '茶':['龍角', '五十嵐', 'Comebuy', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶', '5 Cafe'],
      '豆奶':['5 Cafe'],
      '泡沫':['Comebuy', '政大茶亭', '咖啡大亨'],
      '星冰樂':['星巴克'],
      '啤酒':['5 Cafe'],
      '冰品':['龍角', 'Comebuy', '政大茶亭', '星巴克', '找茶道'],
      '明星':['五十嵐', 'Comebuy', '星巴克', '找茶道'],
      '新品':['五十嵐', 'Comebuy', '星巴克', '找茶道']}

決定 = ['好', '決定', '不錯']
回決定 = ['太棒了' ,'幫到你很開心']

詢問 = ['如何呢？', '可以嗎？', 'ＯＫ嗎？', '可以接受嗎？']

# 沒選擇 = ['隨便', '沒想法', '都可以', '推薦', '建議']

否定 = ['不要', '換', '沒']
建議 = ['有特別想要什麼嗎？', '想要什麼種類呢？', '舉個例吧？']

def 菜單中(message, food):
    for i in food.keys():
        if i in message:
            return i
        
def 食物(name, food):
    print('%s: %s' %('Aっ豆', rn.choice(建議)))
    message = input('%s: ' %(name))
    while not any(i in message for i in 決定):
        if 菜單中(message, food):
            print('%s: 可以參考 %s, %s' %('Aっ豆', rn.choice(food[菜單中(message,food)]), rn.choice(詢問)))
        elif (not 菜單中(message, food)) and (not any(i in message for i in 否定)) :
            print('%s: 好像沒有耶！要不要參考 %s呢？' %('Aっ豆', rn.choice(food[rn.choice(list(food.keys()))])))
        else:
            print('%s: %s %s' %('Aっ豆', rn.choice(food[rn.choice(list(food.keys()))]), rn.choice(詢問)))
        message = input('%s: ' %(name))
    print('%s: %s %s' %('Aっ豆', rn.choice(回決定), rn.choice(繼續)))

In [ ]:
# 天氣
def weather(name):
    url = "https://tw.news.yahoo.com/weather/%E8%87%BA%E7%81%A3/%E8%87%BA%E5%8C%97%E5%B8%82/%E8%87%BA%E5%8C%97%E5%B8%82-2306179"
    #url = "https://tw.news.yahoo.com/weather/%E5%8F%B0%E7%81%A3/%E6%96%B0%E5%8C%97%E5%B8%82/%E6%96%B0%E5%8C%97%E5%B8%82-90717580"
    response = requests.get(url)
    html = response.text
    page = etree.HTML(html)
    print('%s: 這是我幫你查的天氣！' %('Aっ豆'))
    lo=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[1]/div[1]/div/h1//text()')
    t=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[3]/span[1]//text()')
    h=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[2]/span[1]//text()')
    l=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[2]/span[2]//text()')
    r=page.xpath('//*[@id="weather-precipitation"]/div/div[4]/span/span//text()')
    print('         現在溫度 %s : %s 度' %(lo[0],t[0]))
    print('         今日最高溫為 %s 度' %h[0])
    print('         今日最低溫為 %s 度' %l[0])
    print('         今日下雨機率 : %s%s' %(r[0],r[1]))
    p=int(r[0])
    if(p<20):
        print('        天氣不錯呢!')
    elif (20<p<60):
        print('         可能會下雨喔!')
    else:
        print('         下雨機率有點高ㄟ!記得出門要帶傘喔!!')
    print('%s: 要進入網頁嗎?' %('Aっ豆'))
    y = input('%s: ' %(name))
    if("不" not in y):
        a = webbrowser.get('windows-default')
    #a=webbrowser.MacOSX('safari')
        a.open("https://tw.news.yahoo.com/weather/%E8%87%BA%E7%81%A3/%E8%87%BA%E5%8C%97%E5%B8%82/%E8%87%BA%E5%8C%97%E5%B8%82-2306179")
    print("%s :知道天氣就可以安心出門囉!" %('Aっ豆'))

In [21]:
message = ''
name = input('%s: %s %s 你叫什麼名字呀？' %('Aっ豆', rn.choice(歡迎), 問好()))
print('%s: %s' %('Aっ豆', rn.choice(回歡迎)))
message = input('%s: ' %(name))
while not any(i in message for i in 離開):
    if any(i in message for i in 感覺):
        print('%s: %s %s' %('Aっ豆', rn.choice(回感覺), rn.choice(繼續)))
    elif any(i in message for i in 壞心情):
        print('%s: %s %s' %('Aっ豆', rn.choice(回壞心情), rn.choice(繼續)))
    elif any(i in message for i in 好心情):
        print('%s: %s %s' %('Aっ豆', rn.choice(回好心情), rn.choice(繼續)))
    elif any(i in message for i in 吃東西):
        食物(name, 餐點)
    elif any(i in message for i in 喝東西):
        食物(name, 飲料)
    elif any(i in message for i in 時間):
        t = time.localtime(time.time())
        print('%s: 現在 %d年 %d月 %d日 星期%d %d點 %d分 %d秒 %s' %('Aっ豆', t[0], t[1], t[2], t[6]+1, t[3], t[4], t[5], rn.choice(繼續)))
    elif any(i in message for i in 天氣):
        weather(name)
    elif any(i in message for i in 感謝):
        print('%s: %s %s' %('Aっ豆', rn.choice(回感謝), rn.choice(繼續)))
    else:
        print('%s: %s %s' %('Aっ豆', rn.choice(敷衍), rn.choice(繼續)))
    message = input('%s: ' %(name))
print('%s: %s %s' %('Aっ豆', rn.choice(回離開), name))

Aっ豆: 嗨！ 好晚了!怎麼還不快睡覺! 你叫什麼名字呀？kk
Aっ豆: 怎麼了嗎？
kk:餓
Aっ豆: 恩恩 多講點
kk:吃
Aっ豆: 舉個例吧？
kk:粥
Aっ豆: kk 可以參考 飯粥哥, ＯＫ嗎？
kk:不要
Aっ豆: kk 飯粥哥 ＯＫ嗎？
kk:不要
Aっ豆: kk 老麵水煎包 ＯＫ嗎？
kk:不要
Aっ豆: kk 東萊品 ＯＫ嗎？
kk:不要
Aっ豆: kk 鄰居家 如何呢？
kk:飯
Aっ豆: 好像沒有耶！kk 要不要參考 香香呢？
kk:炒飯
Aっ豆: kk 可以參考 敏忠, 如何呢？
kk:好
Aっ豆: 幫到你很開心 然後呢？
kk:掰
Aっ豆: 掰掰 kk
